## MONGODB

### IMPORTING LIBRARIES

In [2]:
import pymongo
from pymongo import MongoClient
from pprint import pprint

import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.parser import parse

import math
from passlib.hash import bcrypt

import random
import string

### CREATING A DATABASE

In [3]:
client = MongoClient()
db = client.package_adbms

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'package_adbms')

### IMPORTING DATASETS

In [5]:
reviews = pd.read_csv("C:\\Users\\admin\\Desktop\\Things_to_be_in_HDD\\8th_Sem\\PACKAGE\\ADBMS\\reviews_.csv")
listings = pd.read_csv("C:\\Users\\admin\\Desktop\\Things_to_be_in_HDD\\8th_Sem\\PACKAGE\\ADBMS\\listings_.csv")

In [6]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,50904,31511792,2015-05-06,19482395,Jihae,Karin’s “Aplace” is absolutely beautiful and c...
1,116134,972063,2012-03-05,928644,Aurélien,"Amazing flat, really close from the MAS Musem,..."
2,116134,1344011,2012-05-25,231288,Gail,"This is a well equipped, very comfortable apar..."
3,116134,7019975,2013-09-03,7984251,Marcel,This is a very nice appartement. We really lik...
4,116134,7693157,2013-09-30,1609507,Barbara,"The appartment is beautiful, the best I've eve..."


In [7]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,50904,https://www.airbnb.com/rooms/50904,20200227171200,2020-02-27,aplace/antwerp: cosy suite - fashion district,Decorated in a vintage style combined with a f...,Aplace Antwerp is located on a charming little...,Decorated in a vintage style combined with a f...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,4,2,0,0,0.02
1,116134,https://www.airbnb.com/rooms/116134,20200227171200,2020-02-27,Trendy Vacation Apartment Antwerp,Enjoy your stay at our 4 person apartment in t...,"You'll have the chance to stay in a very nice,...",Enjoy your stay at our 4 person apartment in t...,none,"The area ""`t eilandje"" is located at the old h...",...,t,f,strict_14_with_grace_period,f,f,1,1,0,0,1.02
2,224333,https://www.airbnb.com/rooms/224333,20200227171200,2020-02-27,Large stylish room in 1930s house + garden,"Large bedroom in classic 1930s house. Kitchen,...",My house is too big for me currently to use th...,"Large bedroom in classic 1930s house. Kitchen,...",none,The house is located in one of the most beauti...,...,f,f,moderate,f,f,2,0,2,0,0.01
3,224682,https://www.airbnb.com/rooms/224682,20200227171200,2020-02-27,APARTMENT ROSCAM - OLD CENTRE ANTWERP,NaN,"Apartment ""Roscam"" is a charming self catering...","Apartment ""Roscam"" is a charming self catering...",none,There is a paid parking lot around the corner....,...,f,f,moderate,f,f,1,1,0,0,3.38
4,345959,https://www.airbnb.com/rooms/345959,20200227171200,2020-02-27,Marleen's home in Antwerp city,"your entire, private groundfloor 2-bedroom apa...",We welcome you on the ground floor of our hous...,"your entire, private groundfloor 2-bedroom apa...",none,NaN,...,f,f,strict_14_with_grace_period,f,f,2,2,0,0,0.57


In [5]:
listings = listings.replace(to_replace ="strict_14_with_grace_period", 
                 value ="strict") 

In [6]:
listings = listings.replace(to_replace = "Entire home/apt", value = "Entire home")

In [7]:
listings["cancellation_policy"].value_counts()

flexible    854
moderate    822
strict      771
Name: cancellation_policy, dtype: int64

In [8]:
listings["room_type"].value_counts()

Entire home     1862
Private room     542
Shared room       26
Hotel room        17
Name: room_type, dtype: int64

In [9]:
def Generate_Password(stringLength = random.randint(5,9)):
    password_characters = string.ascii_letters + string.digits + string.punctuation
    return(''.join(random.choice(password_characters) for i in range(stringLength)))

In [10]:
passwords = []
for i in range(len(listings)):
    passwords.append(Generate_Password())
listings["password"] = passwords

In [11]:
listings["password"].head()

0    nO,)YoC
1    j7xC)Ag
2    uG*[QhQ
3    tC4tv$r
4    {Wiy?</
Name: password, dtype: object

In [12]:
passwords = []
for i in range(len(reviews)):
    passwords.append(Generate_Password())
reviews["password"] = passwords

In [13]:
# User_Node
db.users.create_index('userid', unique=True)
for index, row in reviews.iterrows():
    user = {
        "userid" : str(row["reviewer_id"]),
        "username" : row["reviewer_name"],
        "password" : str(row["password"])
    }
    try:
        db.users.insert_one(user)
    except pymongo.errors.DuplicateKeyError:
        pass

In [14]:
# Listings_node
db.listings.create_index('listing_id', unique=True)
for index, row in listings.iterrows():
    listing = {
        "listing_id"                  : str(row["id"]),
        "listing_name"                : row["name"],
        "summary"                     : row["summary"],
        "picture_url"                 : row["picture_url"],
        
        "host_id"                     : str(row["host_id"]),
        
        "property_type"               : row["property_type"],
        "room_type"                   : row["room_type"],
        "bed_type"                    : row["bed_type"],
        "no_of_beds"                  : "info NA" if str(type(row["beds"])) == "<class 'float'>" else int(row["beds"]),
        "no_of_bedrooms"              : row["bedrooms"],
        "no_of_accommodates"          : row["accommodates"],
        
        "price"                       : float(row["price"][1:].replace(",", "")),
        "cleaning_fee"                : "info NA" if str(type(row["cleaning_fee"])) == "<class 'float'>" else float(row["cleaning_fee"][1:].replace(",", "")),
        "security_deposit"            : "info NA" if str(type(row["security_deposit"])) == "<class 'float'>" else float(row["security_deposit"][1:].replace(",", "")),
        "monthly_price"               : "info NA" if str(type(row["monthly_price"])) == "<class 'float'>" else float(row["monthly_price"][1:].replace(",", "")),
        "weekly_price"                : "info NA" if str(type(row["weekly_price"])) == "<class 'float'>" else float(row["weekly_price"][1:].replace(",", "")),
        
        "cancellation_policy"         : row["cancellation_policy"],
        "instant_bookable"            : row["instant_bookable"],
            
            "availability_30"         : row["availability_30"],
            "availability_60"         : row["availability_60"],
            "availability_90"         : row["availability_90"],
            "availability_365"        : row["availability_365"],
        
        "neighbourhood" : row["neighbourhood_cleansed"]
        
    }
    
    try:
        db.listings.insert_one(listing)
    except pymongo.errors.DuplicateKeyError:
        pass

In [23]:
db.addresses.create_index('neighbourhood', unique=True)
for index, row in listings.iterrows():
    address = {
        "neighbourhood"      : row["neighbourhood_cleansed"],
        "zipcode"            : row["zipcode"],
        "country_code"       : row["country_code"],
        "coordinates"        : [row["latitude"], row["longitude"]],
        "is_location_exact"  : row["is_location_exact"]
    }
    try:
        db.addresses.insert_one(address)
    except pymongo.errors.DuplicateKeyError:
        pass

In [26]:
for index, row in reviews.iterrows():
    review = {
        "reviewer_id"    : str(row["reviewer_id"]),
        "listing_id"     : str(row["listing_id"]),
        "date"           : datetime.strptime(row["date"], "%Y-%m-%d"),
        "comments"       : row["comments"]
    }
    
    db.reviews.insert_one(review)

In [27]:
# Host Node
db.hosts.create_index('host_id', unique=True)
for index, row in listings.iterrows():
    host = {
        "host_id"             : str(row["host_id"]),
        "host_name"                : row["host_name"],
        "password"           : row["password"],
        "host_image"          : row["host_picture_url"],
        "about"               : row["host_about"],
        
        "listings_count"      : 0 if str(type(row["host_listings_count"])) == "<class 'float'>" else int(row["host_listings_count"]),
        
        "response_time"       : row["host_response_time"],
        "acceptance_rate"     : row["host_acceptance_rate"],
        "response_rate"       : row["host_response_rate"],
        
        "host_since"          : "not available" if str(type(row["host_listings_count"])) == "<class 'float'>" else datetime.strptime(row["host_since"], "%Y-%m-%d"),
        "host_url"            : row["host_url"],
        
        "is_superhost"        : row["host_is_superhost"],
        "host_location"       : row["host_location"],
        
        "identity_verified"   : row["host_identity_verified"] == 'f'      
    }
    try:
        db.hosts.insert_one(host)
    except pymongo.errors.DuplicateKeyError:
        pass

## NEO4J

### IMPORTING LIBRARIES

In [28]:
from neo4j import GraphDatabase

In [29]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "rachsam99"), encrypted = False)
db_session = driver.session()

### CREATING NODES

In [30]:
# User Node

query1 = """CALL apoc.mongodb.get('localhost:27017', 'package_adbms', 'users', null, true) yield value as doc 
CREATE (u: User {id : doc.userid, name : doc.username, password : doc.password})"""

db_session.run(query1)

In [35]:
# Listings Node

query2 = """CALL apoc.mongodb.get('localhost:27017', 'package_adbms', 'listings', null, true) yield value as doc 
CREATE (l : Listing {listing_id : doc.listing_id, listing_name : doc.listing_name, host_id : doc.host_id, 
neighbourhood : doc.neighbourhood})"""
db_session.run(query2)

In [32]:
# Address Node
query3 = """CALL apoc.mongodb.get('localhost:27017', 'package_adbms', 'addresses', null, true) yield value as doc 
CREATE (a : Address {neighbourhood : doc.neighbourhood, zipcode : doc.zipcode, 
country_code : doc.country_code, coordinates : doc.coordinates, is_location_exact : doc.is_location_exact})"""
db_session.run(query3)

In [33]:
# Review Node
query4 = """CALL apoc.mongodb.get('localhost:27017', 'package_adbms', 'reviews', null, true) yield value as doc 
CREATE (r: Review {reviewer_id : doc.reviewer_id, listing_id : doc.listing_id, date : date(doc.date), comments : doc.comments})"""

db_session.run(query4)

In [34]:
#Host Node
query5 = """CALL apoc.mongodb.get('localhost:27017', 'package_adbms', 'hosts', null, true) yield value as doc 
CREATE (h: Hosts {host_id : doc.host_id, host_name : doc.host_name, password : doc.password})"""

db_session.run(query5)

### CREATING RELATIONSHIPS

In [36]:
# Creating Relationship between User and Review
query6 = "MATCH (u: User), (r: Review) WHERE u.id = r.reviewer_id CREATE (u) -[:WROTE]-> (r)"
db_session.run(query6)

In [37]:
# Creating Relationship between Review and Listing
query7 = "MATCH (r: Review), (l: Listing) WHERE r.listing_id = l.listing_id CREATE (r) -[:REVIEWED]-> (l)"
db_session.run(query7)

In [38]:
# Creating Relationship between Listing and Address
query8 = "MATCH (a: Address), (l: Listing) WHERE a.neighbourhood = l.neighbourhood CREATE (l) -[:LOCATED]-> (a)"
db_session.run(query8)

In [40]:
# Creating Relationship between Host and Listing
query9 = "MATCH (h: Hosts), (l: Listing) WHERE h.host_id = l.host_id CREATE (h) -[:HOSTS]-> (l)"
db_session.run(query9)